In [19]:
from personal_twitter_data import TweetLoader
from mongo_data import MongoStore
import pymongo

In [20]:
loader = TweetLoader()
loader.extract_tweets(count=10)

In [21]:
len(loader.get_loaded_tweets_as_json())

10

In [22]:
for tweet in loader.tweets:
    print(tweet.author.name)
    # print(tweet._json)

The Wall Street Journal
Fox News
Boston Celtics
Forbes
Bloomberg
Fox News
Forbes
Bloomberg
The Wall Street Journal
Bloomberg


In [23]:
store = MongoStore("twitter_dashboard_db", "home_timeline")

In [26]:
# tweets = [
#     {"user": "Jason", "id": 1, "text": "Hello"},
#     {"user": "Brandon", "id": 2, "text": "World"},
# ]

for i, tweet in enumerate(loader.tweets):
    try:
        user = tweet.user
        print(f"{i+1}. {user}")
        store.collection.insert_one(
            tweet._json
        )
    except pymongo.errors.DuplicateKeyError:
        print("Duplicate tweet")

1. User(_api=<tweepy.api.API object at 0x11058adf0>, _json={'id': 3108351, 'id_str': '3108351', 'name': 'The Wall Street Journal', 'screen_name': 'WSJ', 'location': 'New York, NY', 'description': 'Sign up for our newsletters and alerts: https://t.co/QevH0DLisA | Got a tip? https://t.co/iXIigdOLPr | For WSJ customer support: https://t.co/DZgH9n4vAI', 'url': 'https://t.co/9rMrYLEXTt', 'entities': {'url': {'urls': [{'url': 'https://t.co/9rMrYLEXTt', 'expanded_url': 'http://wsj.com', 'display_url': 'wsj.com', 'indices': [0, 23]}]}, 'description': {'urls': [{'url': 'https://t.co/QevH0DLisA', 'expanded_url': 'http://wsj.com/newsletters', 'display_url': 'wsj.com/newsletters', 'indices': [40, 63]}, {'url': 'https://t.co/iXIigdOLPr', 'expanded_url': 'http://wsj.com/tips', 'display_url': 'wsj.com/tips', 'indices': [77, 100]}, {'url': 'https://t.co/DZgH9n4vAI', 'expanded_url': 'http://customercenter.wsj.com', 'display_url': 'customercenter.wsj.com', 'indices': [129, 152]}]}}, 'protected': False, 

In [31]:
store.collection.create_index([("id", pymongo.ASCENDING)], name="tweet_id", unique=True, default_language="english")

'tweet_id'

In [32]:
store.save_tweets_to_db(loader.tweets)

1. business


DuplicateKeyError: E11000 duplicate key error collection: twitter_dashboard_db.home_timeline index: _id_ dup key: { _id: ObjectId('63a6ffc7a6d2371ecb53b063') }, full error: {'index': 0, 'code': 11000, 'errmsg': "E11000 duplicate key error collection: twitter_dashboard_db.home_timeline index: _id_ dup key: { _id: ObjectId('63a6ffc7a6d2371ecb53b063') }", 'keyPattern': {'_id': 1}, 'keyValue': {'_id': ObjectId('63a6ffc7a6d2371ecb53b063')}}

In [4]:
# data = store.load_data()
data = store.load_flat_data()
len(data)

2

In [ ]:
data = store.load_data(latest=True)

for user_data in data:
    print(user_data["user"])
    print(len(user_data["tweets"]))
    for tweet in user_data["tweets"]:
        print(tweet["created_at"])

In [ ]:
from datetime import datetime

dt = data[0]["tweets"][0]["created_at"]
tokens = dt.split()
date_info = tokens[1:4] + tokens[5:]
date_info = " ".join(date_info)
date_info

In [ ]:
datetime.strptime(date_info, "%b %d %H:%M:%S %Y")

In [ ]:
d1 = datetime(1,1,1,1,1,1)
d2 = datetime(1,1,1,1,1,2)
l = [d2, d1]
sorted(l)